In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url_base = "https://www.fernuni-hagen.de"
url_overview = "/mi/studium/msc_informatik/module.shtml"

In [3]:
response = requests.get(url_base + url_overview)
response.raise_for_status()

In [4]:
html_context = response.text
soup = BeautifulSoup(html_context, "html.parser")

## Extract modules per catalogue

In [5]:
module_link_dict = {}

for h3 in soup.find_all('h3'):
    if ("Katalog B" in h3.text):
        ul_tag = h3.find_next_sibling("ul")
        if ul_tag:
            li_tags = ul_tag.find_all("li")
            if li_tags:
                for li_tag in li_tags:
                    a_tags = li_tag.find_all("a")

                    for a_tag in a_tags:
                        href_url = a_tag["href"]
                        
                        module_link_dict[f"{h3.text} - {a_tag.text}"] = href_url
    elif ("Katalog M:" in h3.text):
        print(h3)
        ul_tag = h3.find_next_sibling("ul")
        if ul_tag:
            li_tags = ul_tag.find_all("li")
            if li_tags:
                for li_tag in li_tags:
                    a_tags = li_tag.find_all("a")

                    for a_tag in a_tags:
                        href_url = a_tag["href"]
                        
                        module_link_dict[f"{h3.text} - {a_tag.text}"] = href_url

<h3>Katalog M: Theoretische Informatik</h3>
<h3>Katalog M: Technische Informatik</h3>
<h3>Katalog M: Praktische Informatik</h3>


## Extract relevant data

In [6]:
def get_info_module(module, path):
    """
    This function extracts following information for each module:
    - Frequency (summer and/ or winter)
    - Exam type
    - Prerequesites to take the exam
    """
    path = url_base + path
    response = requests.get(path)
    response.raise_for_status()
    html_context = response.text
    soup = BeautifulSoup(html_context, "html.parser")

    info_dict = {}

    for th in soup.find_all('th'):
        if ("Häufigkeit des Moduls" in th):
            td_tag = th.find_next_sibling("td")
            if td_tag:
                #print(th.text, td_tag.text)
                info_dict["Haeufigkeit"] = td_tag.text
        elif ("Art der Prüfungsleistung" in th):
            td_tag = th.find_next_sibling("td")
            if td_tag:
                #print(th.text, td_tag.text)
                info_dict["Pruefung"] = td_tag.text
        elif ("Voraussetzung" in th):
            td_tag = th.find_next_sibling("td")
            if td_tag:
                #print(th.text, td_tag.text)
                info_dict["Voraussetzung"] = td_tag.text
    return info_dict

In [7]:
moduls_dict = {}

for module, path in module_link_dict.items():
    moduls_dict[module] = get_info_module(module, path)
    print(f"{module} DONE")
    print("--")
    time.sleep(1)

Katalog B - 61412 Lineare Optimierung DONE
--
Katalog B - 63112 Übersetzerbau DONE
--
Katalog B - 63117 Data Mining DONE
--
Katalog B - 63122 Architektur und Implementierung von Datenbanksystemen DONE
--
Katalog B - 63211 Verteilte Systeme DONE
--
Katalog B - 63311 Einführung in Mensch-Computer-Interaktion DONE
--
Katalog B - 63312 Interaktive Systeme DONE
--
Katalog B - 63514 Simulation DONE
--
Katalog B - 63711 Anwendungsorientierte Mikroprozessoren DONE
--
Katalog B - 63712 Parallel Programming DONE
--
Katalog B - 64111 Betriebliche Informationssysteme DONE
--
Katalog B - 64112 Entscheidungsmethoden in unternehmensweiten Softwaresystemen DONE
--
Katalog B - 64211 Wissensbasierte Systeme DONE
--
Katalog B - 64313 Mobile Security DONE
--
Katalog M: Theoretische Informatik - 61115 Mathematische Grundlagen der Kryptografie DONE
--
Katalog M: Theoretische Informatik - 61414 Effiziente Graphenalgorithmen DONE
--
Katalog M: Theoretische Informatik - 63213 Algorithmische Geometrie DONE
--
K

In [8]:
for k, v in moduls_dict.items():
    print(k, v)

Katalog B - 61412 Lineare Optimierung {'Haeufigkeit': 'in jedem Sommersemester', 'Pruefung': 'benotete mündliche Prüfung', 'Voraussetzung': 'keine'}
Katalog B - 63112 Übersetzerbau {'Haeufigkeit': 'in jedem Wintersemester', 'Pruefung': 'benotete mündliche Prüfung', 'Voraussetzung': 'keine'}
Katalog B - 63117 Data Mining {'Haeufigkeit': 'in jedem Semester', 'Pruefung': 'benotete Prüfungsklausur', 'Voraussetzung': 'keine'}
Katalog B - 63122 Architektur und Implementierung von Datenbanksystemen {'Haeufigkeit': 'in jedem Sommersemester', 'Pruefung': 'benotete mündliche Prüfung', 'Voraussetzung': 'keine'}
Katalog B - 63211 Verteilte Systeme {'Haeufigkeit': 'in jedem Wintersemester', 'Pruefung': 's. Anmerkung', 'Voraussetzung': 'keine'}
Katalog B - 63311 Einführung in Mensch-Computer-Interaktion {'Haeufigkeit': 'in jedem Semester', 'Pruefung': 'benotete Prüfungsklausur', 'Voraussetzung': 'Als Zulassungsvoraussetzung für die Modulprüfungsklausur müssen bei 6 von 7 Kurseinheiten 75% der Einsen

## Save data

In [11]:
import pandas as pd


data = []

for modul, attributes in moduls_dict.items():
    lst = modul.split("-")
    mod = lst[1].strip()
    kat = lst[0].strip()
    
    row = {'Modul': mod, 'Katalog': kat}
    row.update(attributes)
    data.append(row)

# Create Dataframe
df = pd.DataFrame(data)

In [12]:
df.head()

,Modul,Katalog,Haeufigkeit,Pruefung,Voraussetzung
0,61412 Lineare Optimierung,Katalog B,in jedem Sommersemester,benotete mündliche Prüfung,keine
1,63112 Übersetzerbau,Katalog B,in jedem Wintersemester,benotete mündliche Prüfung,keine
2,63117 Data Mining,Katalog B,in jedem Semester,benotete Prüfungsklausur,keine
3,63122 Architektur und Implementierung von Date...,Katalog B,in jedem Sommersemester,benotete mündliche Prüfung,keine
4,63211 Verteilte Systeme,Katalog B,in jedem Wintersemester,s. Anmerkung,keine


In [ ]:
# Save Dataframe to csv / excel

df.to_csv("Module_Uebersicht.csv")
# df.to_excel("Module_Uebersicht.xlsx", index=False)

In [ ]:
# Read csv (if no need to update data from Website)

"""
df = pd.read_csv("Module_Uebersicht.csv", index_col=False)

"""